<a href="https://colab.research.google.com/github/Omars2003/HuggingFace/blob/main/ResumenParte3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers #instalamos los transformadores para poder crear nuestro modelo  
!pip install transformers[sentencepiece]
!pip install datasets #instalamos datasets para poder descargar la base de datos de hf con l a que vamos a entrenar nuestro modelo
!pip install rouge_score #instalamos rouge_score que sera capaz de decirnos que tan bien se esta haciendo la funcion del resumen(esta es especifica para poder calificar task como el resumen)
!pip install evaluate 
!pip install nltk
 #natural lenguage toolkit, con este podremos tokenizar 
!pip install accelerate

In [2]:
from huggingface_hub import notebook_login #para poder guardar nuestro modelo directamente a nuestro git de huggingface
from datasets import load_dataset # se importa la funcion de cargar datos al programa en este caso google colab
from datasets import concatenate_datasets, DatasetDict #concatenate datasets  va  a hacer que los datos en ingles y los de espeañol se pueden juntar en uno solo para hacer mas facil el task
from transformers import AutoTokenizer #aqui ocupamos AutoTokenizer para que nuestro checkpoint pueda manipular o sea tokenizar la dataset
import evaluate
import nltk
import pandas as pd
from transformers import AutoModelForSeq2SeqLM #este modelo es sequence2sequence que tiene las funciones de un encoder-decoder que se usa en este tipo de task
from transformers import Seq2SeqTrainingArguments #como se entrentaran textos para poder crear un buen modelo que hace buenos resumenes con este se entrenaran este tipo de datos
import numpy as np
from transformers import DataCollatorForSeq2Seq #aqui ocupamos un data collator que formara un batch usando los datos de train y validation, se ocuparan para el paddin y cosas asii
from transformers import Seq2SeqTrainer # con este objeto entrenamos nuestro modelo de S2S antes de poder hacerlo con una interfas

In [3]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [4]:
spanish_dataset = load_dataset("amazon_reviews_multi", "es") #aqui definimos una variable que contendra todos los datos en español de nuestro data set
english_dataset = load_dataset("amazon_reviews_multi", "en")# aqui una variable pero con los datos en ingles
#para este proyecto ocuparemos un dataset de reviews de amazon,estan en otros idiomas, en este caso solo lo uqeremos en ingles y español

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#para hacer nuestro programa solo nos interesa de los datos el titulo del review y su contenido, por lo que solo ocuparemos esos datos
def muestra_ejemplos(dataset, num_samples=4, seed=50):#seed=un grupo de 50,creamos un atributo que usara el dataset,los ejemplos que queremos visualizar y entre cuantos ejemplos queremos que nos muestre el titulo y el review
    ejemplos = dataset["train"].shuffle(seed=seed).select(range(num_samples)) #shuffle funtion cambia el orden,definimos la variable ejemplos, expresandole que:dentro de los datos que estan en la agrupacion de train, unos 50 ejemplos les cambies el orden y que en un rango de 4 eligas 4 
    for example in ejemplos: #aqui hacemos un arreglo para que imprima 4 ejemplos de manera random
        print(f"\n'>> Title: {example['review_title']}'") #aqui imprime el titulo
        print(f"'>> Review: {example['review_body']}'")#aqui imprime la reseña como tal


muestra_ejemplos(english_dataset) #aqui nos nuestra el atributo usando los datos en ingles
muestra_ejemplos(spanish_dataset)#y aqui usando los datos en español

In [ ]:
english_dataset.set_format("arrow") #arrow or #pandas #aqui en formato arrow podemos ver el tipo de producto y el numero de reseñas 
english_df = english_dataset["train"][:] #aqui podemos ver toda la info en ingles, tambien lo podriamos hacer con los datos en español y no tendriamos problemas
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20] #aqui nos enseña la categoria  de solo 20 departamentos

In [12]:
#solo usaremos la informacion de titulos y reviews de los libros, por lo que vamos a filtrar la informacion para que solo se use esta
def filtros_books(example):  #creamos un objeto que haga la funcion de filtrar libros ya que estan los normales y los e -books 
    return ( #nos regresa
        example["product_category"] == "book" #la categoria de libros
        or example["product_category"] == "digital_ebook_purchase" )


In [13]:
english_dataset.reset_format() #cambiamos la estructura de los datos 
spanish_dataset.reset_format()
spanish_books = spanish_dataset.filter(filtros_books) #ahora las variables que tenian los datos en ingles ya estan filtrados y solo tienen la informacion de los libros
english_books = english_dataset.filter(filtros_books)#lo mismo con la de ingles
muestra_ejemplos(english_books)#imprimos para comprobar que si se tarete de textos referentes a libros
muestra_ejemplos(spanish_books)# lo mismo aqui

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]


'>> Title: Steamy!!'
'>> Review: This was a great book. I loved the characters and there was a story line that you could believe. And the steamy scenes, let's just say they were scorching hot.'

'>> Title: Great info for 4 year old.'
'>> Review: 4 year old grandson loves the pictures and can almost recite all the text when reading it alone. Well done in deed'

'>> Title: Love this'
'>> Review: This book is a amazing book for connect your relation. It can help you to maintaining your relationship.this book save you to harm effect of relation.i love it......'

'>> Title: Excellent series'
'>> Review: I read this series once and will again and again.'

'>> Title: Kakebo, está genial'
'>> Review: Ya sabía lo que compraba. Tuve otro justo antes. Los gastos están bien clasificados. Me ayuda a ser consciente de mis gastos'

'>> Title: Ojo fijaros bien en el idioma antes d comprarlos'
'>> Review: Los pedi en catalán por error. Intenté anular el pedido dos minutos más tarde para pedirlos en ca

In [14]:
books_dataset = DatasetDict() #se creara una nueva variable que se indicara que dividira informacion y la guardara en una sola
for split in english_books.keys():#
    books_dataset[split] = concatenate_datasets( #en la variaable se concatenan las dos datasets y se hace solo una
        [english_books[split], spanish_books[split]]
    )
books_dataset[split] = books_dataset[split].shuffle(seed=60) # a esta variable ya concatenada con info en spañol e ingles la mezclamos para que nuestro algoritmo no aprenda mas de un idioma que de otro.

In [ ]:
muestra_ejemplos(books_dataset) #vemos unos ejemplos!

In [ ]:
books_dataset = books_dataset.filter(lambda x: len(x["review_title"].split()) > 2) #los datos que tengan 2 o menos palabras las vamos a quitar ya que no nos serviran de mucho para entrenar y hacer un buen algoritmo de resumen.

In [17]:
model_checkpoint = "google/mt5-small" #aqui ya vamos a cargar nuestro modelo que emepzara a trabajar con los datos y emepzar con la funcion de summarization
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)# el tokenizer ya modifica la data, usamos el checkpoint google/mt5 un big data que haci sido entrenado para trabajos textTo text haciendo que nuestro modelo tenga un buen desarrollo

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
input=tokenizer("I have finished How the world really works") #aqui podemos ver un ejemplo de como tokeniza este checkpoiint le da valores numericos a inputs para poder hacer un proceso computacional adecuado
input

In [19]:
tokenizer.convert_ids_to_tokens(input.input_ids) #aqui convertimos los numeros que se llaman ids a tokens y podemos ver como los separa por palabra y por puntuaciones
#al ser un checkpoint que sirve para muchisimos idiomas su forma de tokenizar las palabras dse adapta a la mayoria de los idiomas

['▁I',
 '▁have',
 '▁',
 'finished',
 '▁How',
 '▁the',
 '▁world',
 '▁',
 'really',
 '▁works',
 '</s>']

In [20]:
max_input_length = 500 # de lamos un maximo de valor a la entrada 
max_target_length = 30 #y un valor maximo a como queremos que sea nuestro target o el resumen, lo puede modificar cadar programador
def preprocess_function(examples): #creamos atributo que tokenize al mismo tiempo el input y el target, esto para hacer mejor nuestro algoritmo
    model_inputs = tokenizer(
        examples["review_body"], max_length=max_input_length, truncation=True #tokenizar ejemplos del review_body dandoles el maximo que pusimos y aplicamos truncacion para que todos tengas el mismo valor osea el maximo(600)
    )
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer(): 
        labels = tokenizer(
            examples["review_title"], max_length=max_target_length, truncation=True #ahora tokeniza el titulo y le da un maximo de 30 y hace que todos aun que sean mayores  se queden en en 30
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = books_dataset.map(preprocess_function, batched=True) #aqui ya se tokeniza titulo y cuerpo a la par de todo el dataset de books, y se rellenan con el batch
tokenized_datasets
#aqui se aplican los parametros establecidos a TODOS nuestros datos con la funcion .map dandoles simetria con el batch, podemos ver los datos que nos quedan con sus columnas

In [22]:
rouge_score = evaluate.load("rouge") #instalamos rouge que va a ser nuestro calificador de que tan bien se esta haciendo nuestro resumen
nltk.download("punkt") #This tokenizer divides a text into a list of sentences
#by using an unsupervised algorithm to build a model for abbreviation
#words, collocations, and words that start sentences.  It must be
#trained on a large collection of plaintext in the target language
#before it can be used.
#este tokenizer nos va a yudar ya que al no ser supervisado ve los . palabras y se va entrenando para que pueda leer textos con diferentes caracteres y no pierda su funcion ya que
#con otros puede llegar a perder inteligencia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:

from nltk.tokenize import sent_tokenize 
def three_sentence_summary(text):#creamos un atributo que nos de 3 oraciones y que con estas se pueda comparar nuestro resumen
   return "\n".join(sent_tokenize(text)[:3]) #aqui regresa ya 3 oraciones del texto


print(three_sentence_summary(books_dataset["train"][6]["review_body"]))# imprimos informacion de los reviews del los datos de train, aplicandole la tokenizacion de 3_sentence_

....like just about every other page!!
I couldn't believe when I started reading this (with Book 1) and around the fifth chapter, things started feeling freaky.
Dialog didn't continue, descriptions faltered, and why?


In [24]:
def evaluate_baseline(dataset, metric):#Something, such as a set of data, used as a basis for comparison or as a control in a study
#ocuparemos una baseline para comprarar nuestro resumen, ayuda a que no tenga faltas de ortografia o puntuacion y que tenga coherencia
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]#la variable de summaries compara nltk tokenizador o su info que creo arriba con  datos del review
    return metric.compute(predictions=summaries, references=dataset["review_title"]) #y nos regresa metricas computaciones
    #summaries es igual a las predictiones que es el resumen que nuestro modelo crea ya analiada y mejorara con nltk tecnologia :) yy las referencias las saca de nuestros datasets de revie_title
    #las referencia normalmente son creadas por umanos asi puede comparar y analizar el lenguaje en el que se nuestro modelo crea todo

    #d ROUGE is based on computing the precision and recall scores for the overlap.
#Recall= Totalnumberofwords /  inreferencesummaryNumberofoverlappingwords #el numero de palabras que son las mismas entre si 
#Precision= Totalnumberofwordsingeneratedsummary/ Numberofoverlappingwords #el numero de palabras que hay en las de referencia sobre las que nuestro modelo gener

In [25]:
#aqui ya usaremos las biblioteca de rouge que seran las encargadas en analizar que tan bien ha sido creado nuestro resumen, analizandolo con diferentes parametros
score = evaluate_baseline(books_dataset["validation"], rouge_score) #esta variable guardara la puntuacion  evaluando nuestro baseline con datos creados por humanos que estan en el set de datos de validacion!y lo calificara usando el rouge score
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"] #una cadena con los nombres de los diferentes tipos de calificados, o como yo los llamo profesores
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names) # un contador en el que se califica nuestra baseline con cada tipo de profesor
rouge_dict# lo imprimimos
#la forma que rouge funciona es que el 1 separa cada palabra y ve cuantas se repiten la segunda agarr dos paralbaras y asi va agarrando mas grupos de textos y ver cuantas palabras iguales tiene y con eso da los resultados

{'rouge1': 16.75, 'rouge2': 8.86, 'rougeL': 15.57, 'rougeLsum': 15.96}

In [45]:
#now lets fine-tune our model!!
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)#We’ll need to generate summaries in order to compute ROUGE scores during training.
#transformers include a couple of classes which are going to hel us with this tas



In [47]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [49]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_datasets["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-amazon-en-es-Resumen-2",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
)

In [50]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [51]:

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)


{'input_ids': tensor([[  4927,  97789,   2888,   1070,   5835,    527,    287,   2250,   2225,
         105255,    910,    269, 202133,  62799,    260,    259,   9918,    336,
            259,  68140,    287,  10945,    339,    259,    262, 123647,    260,
            298,  53799,    400,  55976,    260,      1],
        [   336,    259,  91451,    714,   3435,    351,    259,   4293,   4668,
           1385,   8103,  24938,    285,    260,      1,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[  2368,   4267,    714,   3435,    259,   2220,    272,    293,    270,
           3162,      1],
        [

In [52]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/OMARS200/mt5-small-finetuned-amazon-en-es-Resumen-2 into local empty directory.


In [53]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9672
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9672


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,7.004000,3.340821,13.387400,5.282800,12.981200,12.966600
2,3.903600,3.228628,15.915800,7.759100,15.286300,15.393200
3,3.598000,3.177768,16.062800,7.117800,15.122700,15.248700
4,3.422800,3.114703,15.999800,7.623900,15.369100,15.471100
5,3.321200,3.104929,16.536900,7.743000,16.227200,16.342700
6,3.265800,3.097401,17.412500,8.096100,17.022300,17.136000
7,3.210500,3.091120,16.578600,7.736900,16.188900,16.318900
8,3.184000,3.091032,16.379900,7.608800,15.988600,16.169100


Saving model checkpoint to mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500
Configuration saved in mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500/config.json
Model weights saved in mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-500/spiece.model
Special tokens file saved in mt5-small-finetuned-amazon-en-es-Resumen-2/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-amazon-en-es-Resumen-2/spiece.model
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.
Saving model checkpoint to mt5-small-finetuned-amazon-en-es-Resumen-2/checkpoint-1000
Configuration saved in mt5-small-finetuned-amazon

TrainOutput(global_step=9672, training_loss=3.863758020897953, metrics={'train_runtime': 2051.8558, 'train_samples_per_second': 37.71, 'train_steps_per_second': 4.714, 'total_flos': 1.187876662960128e+16, 'train_loss': 3.863758020897953, 'epoch': 8.0})

In [54]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

Saving model checkpoint to mt5-small-finetuned-amazon-en-es-Resumen-2
Configuration saved in mt5-small-finetuned-amazon-en-es-Resumen-2/config.json
Model weights saved in mt5-small-finetuned-amazon-en-es-Resumen-2/pytorch_model.bin
tokenizer config file saved in mt5-small-finetuned-amazon-en-es-Resumen-2/tokenizer_config.json
Special tokens file saved in mt5-small-finetuned-amazon-en-es-Resumen-2/special_tokens_map.json
Copy vocab file to mt5-small-finetuned-amazon-en-es-Resumen-2/spiece.model
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/1.12G [00:00<?, ?B/s]

Upload file runs/Aug02_03-06-56_5bb3b30204d5/events.out.tfevents.1659409822.5bb3b30204d5.81.0:  37%|###6      …

To https://huggingface.co/OMARS200/mt5-small-finetuned-amazon-en-es-Resumen-2
   bffa7d1..19772b9  main -> main

   bffa7d1..19772b9  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 16.3799}]}
To https://huggingface.co/OMARS200/mt5-small-finetuned-amazon-en-es-Resumen-2
   19772b9..b58717e  main -> main

   19772b9..b58717e  main -> main



'https://huggingface.co/OMARS200/mt5-small-finetuned-amazon-en-es-Resumen-2/commit/19772b9aa9748d9dcbd813c2e00d44b6911444f0'